# Word2Vec之Skip-Gram模型-中文文本版

下面代码将用TensorFlow实现Word2Vec中的Skip-Gram模型。

关于Skip-Gram模型请参考上一篇[知乎专栏文章](https://zhuanlan.zhihu.com/p/27234078)

# 1 导入包

In [1]:
import time
import numpy as np
import tensorflow as tf
import random
from collections import Counter

# 2 加载数据

数据集使用的是来自Matt Mahoney的维基百科文章，数据集已经被清洗过，去除了特殊符号等，并不是全量数据，只是部分数据，所以实际上最后训练出的结果很一般（语料不够）。

如果想获取更全的语料数据，可以访问以下网站，这是gensim中Word2Vec提供的语料：

- 来自Matt Mahoney预处理后的[文本子集](http://mattmahoney.net/dc/enwik9.zip)，里面包含了10亿个字符。
- 与第一条一样的经过预处理的[文本数据](http://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2)，但是包含了30个亿的字符。
- 多种语言的[训练文本](http://www.statmt.org/wmt11/translation-task.html#download)。
- [UMBC webbase corpus](http://ebiquity.umbc.edu/redirect/to/resource/id/351/UMBC-webbase-corpus)

In [2]:
with open('data/Javasplittedwords') as f:
    text = f.read()

# 3 数据预处理

数据预处理过程主要包括：

- 替换文本中特殊符号并去除低频词
- 对文本分词
- 构建语料
- 单词映射表

In [18]:
# 筛选低频词
words_count = Counter(words)
words = [w for w in words if words_count[w] > 50]

In [19]:
# 构建映射表
vocab = set(words)
vocab_to_int = {w: c for c, w in enumerate(vocab)}
int_to_vocab = {c: w for c, w in enumerate(vocab)}

In [20]:
print("total words: {}".format(len(words)))
print("unique words: {}".format(len(set(words))))

total words: 8623686
unique words: 6791


In [21]:
# 对原文本进行vocab到int的转换
int_words = [vocab_to_int[w] for w in words]

# 4 采样

对停用词进行采样，例如“the”， “of”以及“for”这类单词进行剔除。剔除这些单词以后能够加快我们的训练过程，同时减少训练过程中的噪音。

我们采用以下公式:
$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

其中$ t $是一个阈值参数，一般为1e-3至1e-5。  
$f(w_i)$ 是单词 $w_i$ 在整个数据集中的出现频次。  
$P(w_i)$ 是单词被删除的概率。

>这个公式和论文中描述的那个公式有一些不同

In [22]:
t = 1e-5 # t值
threshold = 0.9 # 剔除概率阈值

# 统计单词出现频次
int_word_counts = Counter(int_words)
total_count = len(int_words)
# 计算单词频率
word_freqs = {w: c/total_count for w, c in int_word_counts.items()}
# 计算被删除的概率
prob_drop = {w: 1 - np.sqrt(t / word_freqs[w]) for w in int_word_counts}
# 对单词进行采样
train_words = [w for w in int_words if prob_drop[w] < threshold]

In [23]:
len(train_words)

3883241

# 5 构造batch

Skip-Gram模型是通过输入词来预测上下文。因此我们要构造我们的训练样本，具体思想请参考知乎专栏，这里不再重复。

对于一个给定词，离它越近的词可能与它越相关，离它越远的词越不相关，这里我们设置窗口大小为5，对于每个训练单词，我们还会在[1:5]之间随机生成一个整数R，用R作为我们最终选择output word的窗口大小。这里之所以多加了一步随机数的窗口重新选择步骤，是为了能够让模型更聚焦于当前input word的邻近词。

In [25]:
def get_targets(words, idx, window_size=5):
    '''
    获得input word的上下文单词列表
    
    参数
    ---
    words: 单词列表
    idx: input word的索引号
    window_size: 窗口大小
    '''
    target_window = np.random.randint(1, window_size+1)
    # 这里要考虑input word前面单词不够的情况
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    # output words(即窗口中的上下文单词)
    targets = set(words[start_point: idx] + words[idx+1: end_point+1])
    return list(targets)

In [26]:
def get_batches(words, batch_size, window_size=5):
    '''
    构造一个获取batch的生成器
    '''
    n_batches = len(words) // batch_size
    
    # 仅取full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx: idx+batch_size]
        for i in range(len(batch)):
            batch_x = batch[i]
            batch_y = get_targets(batch, i, window_size)
            # 由于一个input word会对应多个output word，因此需要长度统一
            x.extend([batch_x]*len(batch_y))
            y.extend(batch_y)
        yield x, y

# 6 构建网络

该部分主要包括：

- 输入层
- Embedding
- Negative Sampling

## 输入

In [27]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None, None], name='labels')

## Embedding

嵌入矩阵的矩阵形状为 $ vocab\_size\times hidden\_units\_size$ 

TensorFlow中的tf.nn.embedding_lookup函数可以实现lookup的计算方式

In [28]:
vocab_size = len(int_to_vocab)
embedding_size = 200 # 嵌入维度

In [29]:
with train_graph.as_default():
    # 嵌入层权重矩阵
    embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    # 实现lookup
    embed = tf.nn.embedding_lookup(embedding, inputs)

## Negative Sampling

负采样主要是为了解决梯度下降计算速度慢的问题，详情同样参考我的上一篇知乎专栏文章。

TensorFlow中的tf.nn.sampled_softmax_loss会在softmax层上进行采样计算损失，计算出的loss要比full softmax loss低。

In [30]:
n_sampled = 100

with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(vocab_size))
    
    # 计算negative sampling下的损失
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, vocab_size)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## 验证

为了更加直观的看到我们训练的结果，我们将查看训练出的相近语义的词。

In [33]:
with train_graph.as_default():
    # 随机挑选一些单词
    ## From Thushan Ganegedara's implementation
    valid_size = 7 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = [vocab_to_int['word'], 
                      vocab_to_int['ppt'], 
                      vocab_to_int['熟悉'],
                      vocab_to_int['java'], 
                      vocab_to_int['能力'], 
                      vocab_to_int['逻辑思维'],
                      vocab_to_int['了解']]
    
    valid_size = len(valid_examples)
    # 验证单词集
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # 计算每个词向量的模并进行单位化
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    # 查找验证单词的词向量
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    # 计算余弦相似度
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [34]:
epochs = 10 # 迭代轮数
batch_size = 1000 # batch大小
window_size = 10 # 窗口大小

with train_graph.as_default():
    saver = tf.train.Saver() # 文件存储

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        # 
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            # 计算相似的词
            if iteration % 1000 == 0:
                # 计算similarity
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # 取最相似单词的前8个
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to [%s]:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
            
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 3.8901 0.1061 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 3.7713 0.0992 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 3.4199 0.0923 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 3.3403 0.1041 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 3.4817 0.1069 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 3.1479 0.1065 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 2.7737 0.1067 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 2.7500 0.1023 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 3.0976 0.1019 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 2.9936 0.0999 sec/batch
Nearest to [word]: 公关, 机构, 诚恳, 革新, 内涵, 培训讲师, 主播, 同行,
Nearest to [ppt]: 官网, 新产品, 汇总, 前期, 小说, 技术开发, 操, 新媒体营销,
Nearest to [熟悉]: 不拘泥, 态度, 考试, 所需, 麻将, 年初, 经纪人, 科目,
Nearest to [java]: 最优, 信用卡, 竞争力, 全力, 录入, 热爱工作, 重庆, 建筑,
Nearest to [能力]: 人事管理, 报警, 上传下达, 成交, 新闻报道, 计算机, 算清, 同理,
Nearest to [逻辑思维]: 绩效考核, 自动控制

Epoch 3/10 Iteration: 7900 Avg. Training loss: 2.1527 0.1104 sec/batch
Epoch 3/10 Iteration: 8000 Avg. Training loss: 2.5490 0.1109 sec/batch
Nearest to [word]: 熟练应用, 诚恳, 条理清晰, 中英文, 版主, 交互性, 交予, 公关,
Nearest to [ppt]: 会计学, 词性, 操, 网络资源, 新产品, 文字处理, 操作, 前期,
Nearest to [熟悉]: 不拘泥, 格式, 所需, 缺陷, 记录, 考试, 麻将, jquery,
Nearest to [java]: 全力, 信用卡, 竞争力, 建筑, 热爱工作, 研发管理, 坚守, 安全,
Nearest to [能力]: 成交, 宝, 新闻报道, 读者, 计算机, 同理, 估值, 人事管理,
Nearest to [逻辑思维]: 自动控制, 语言表达, 登录, 态度端正, 学科, 程序设计, 主人翁, 体系结构,
Nearest to [了解]: 分析方法, 方向和, 求职, 极客, 完整, 专情, 调研, 高强,
Epoch 3/10 Iteration: 8100 Avg. Training loss: 1.8622 0.1001 sec/batch
Epoch 3/10 Iteration: 8200 Avg. Training loss: 2.5565 0.0981 sec/batch
Epoch 3/10 Iteration: 8300 Avg. Training loss: 2.3739 0.1019 sec/batch
Epoch 3/10 Iteration: 8400 Avg. Training loss: 2.3665 0.1049 sec/batch
Epoch 3/10 Iteration: 8500 Avg. Training loss: 2.2349 0.1062 sec/batch
Epoch 3/10 Iteration: 8600 Avg. Training loss: 2.1831 0.1060 sec/batch
Epoch 3/10 Iteration: 8700 Avg. Training l

Epoch 4/10 Iteration: 15100 Avg. Training loss: 2.1128 0.0945 sec/batch
Epoch 4/10 Iteration: 15200 Avg. Training loss: 2.0971 0.0930 sec/batch
Epoch 4/10 Iteration: 15300 Avg. Training loss: 2.2563 0.1008 sec/batch
Epoch 4/10 Iteration: 15400 Avg. Training loss: 2.2727 0.1037 sec/batch
Epoch 4/10 Iteration: 15500 Avg. Training loss: 2.2474 0.1037 sec/batch
Epoch 5/10 Iteration: 15600 Avg. Training loss: 2.1207 0.0701 sec/batch
Epoch 5/10 Iteration: 15700 Avg. Training loss: 2.2184 0.1021 sec/batch
Epoch 5/10 Iteration: 15800 Avg. Training loss: 2.3385 0.1111 sec/batch
Epoch 5/10 Iteration: 15900 Avg. Training loss: 1.8494 0.1084 sec/batch
Epoch 5/10 Iteration: 16000 Avg. Training loss: 2.5302 0.1034 sec/batch
Nearest to [word]: 熟练应用, 诚恳, 平者让, 友善, 公关, 空白, office, 交互性,
Nearest to [ppt]: 网络资源, 操, 操作, 文字处理, 透视, 词性, project, 会计学,
Nearest to [熟悉]: 缺陷, 不拘泥, 格式, 记录, 所需, jquery, 老客户, 考试,
Nearest to [java]: 全力, 坚守, 融入, 秉持, 安全, 竞争力, 信用卡, 热爱工作,
Nearest to [能力]: 宝, 成交, 计算机, 人事管理, 估值, 报警, 受众, 敏锐地,


Epoch 6/10 Iteration: 22700 Avg. Training loss: 2.4528 0.0962 sec/batch
Epoch 6/10 Iteration: 22800 Avg. Training loss: 2.1565 0.0930 sec/batch
Epoch 6/10 Iteration: 22900 Avg. Training loss: 1.9977 0.0989 sec/batch
Epoch 6/10 Iteration: 23000 Avg. Training loss: 2.1165 0.0948 sec/batch
Nearest to [word]: 交互性, 空白, 熟练应用, 制造业, 交予, 友善, 平者让, 软件应用,
Nearest to [ppt]: 网络资源, 操, project, 操作, 透视, 文字处理, 官网, 出品,
Nearest to [熟悉]: 缺陷, 记录, 不拘泥, 格式, 所需, jquery, 老客户, 考试,
Nearest to [java]: 全力, 融入, 秉持, 精美, 坚守, 参照, 热爱工作, 安全,
Nearest to [能力]: 宝, 成交, 计算机, 人事管理, 报警, 估值, 著名, 抓,
Nearest to [逻辑思维]: 自动控制, 登录, 措施, 程序设计, 精准, 语言表达, 态度端正, 有意者,
Nearest to [了解]: 方向和, 求职, 分析方法, 专情, 高强, 下级, 肯吃苦, 函数,
Epoch 6/10 Iteration: 23100 Avg. Training loss: 2.1464 0.0925 sec/batch
Epoch 6/10 Iteration: 23200 Avg. Training loss: 2.2254 0.0935 sec/batch
Epoch 7/10 Iteration: 23300 Avg. Training loss: 2.2035 0.0022 sec/batch
Epoch 7/10 Iteration: 23400 Avg. Training loss: 2.1513 0.1039 sec/batch
Epoch 7/10 Iteration: 23500 Avg. Trai

Epoch 8/10 Iteration: 30100 Avg. Training loss: 2.3232 0.1058 sec/batch
Epoch 8/10 Iteration: 30200 Avg. Training loss: 2.1217 0.1073 sec/batch
Epoch 8/10 Iteration: 30300 Avg. Training loss: 1.9782 0.1065 sec/batch
Epoch 8/10 Iteration: 30400 Avg. Training loss: 2.3834 0.1061 sec/batch
Epoch 8/10 Iteration: 30500 Avg. Training loss: 2.4111 0.1055 sec/batch
Epoch 8/10 Iteration: 30600 Avg. Training loss: 2.0591 0.1050 sec/batch
Epoch 8/10 Iteration: 30700 Avg. Training loss: 2.0199 0.1065 sec/batch
Epoch 8/10 Iteration: 30800 Avg. Training loss: 2.1791 0.1037 sec/batch
Epoch 8/10 Iteration: 30900 Avg. Training loss: 2.1995 0.1136 sec/batch
Epoch 8/10 Iteration: 31000 Avg. Training loss: 2.2319 0.1045 sec/batch
Nearest to [word]: 制造业, 熟练应用, 软件应用, excel, 交予, 平者让, 空白, 交互性,
Nearest to [ppt]: 透视, 操, 文字处理, 网络资源, project, 操作, 官网, 会计学,
Nearest to [熟悉]: 缺陷, 记录, 不拘泥, 格式, jquery, 法律法规, 老客户, 考试,
Nearest to [java]: 全力, 融入, 精美, 秉持, 启动, 安全, 参照, 信用卡,
Nearest to [能力]: 宝, 成交, 计算机, 人事管理, 报警, 棒, 手持, 估值,
N

KeyboardInterrupt: 